In [3]:
#install gradio
!pip install gradio

   ---------------------------------------- 0.0/18.1 MB ? eta -:--:--
   ------------------- -------------------- 8.9/18.1 MB 39.7 MB/s eta 0:00:01
   ---------------------------------------  17.8/18.1 MB 41.7 MB/s eta 0:00:01
   ---------------------------------------- 18.1/18.1 MB 35.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 27.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.2 MB ? eta -:--:--
   ---------------------------------- ----- 9.7/11.2 MB 46.5 MB/s eta 0:00:01
   ---------------------------------------- 11.2/11.2 MB 31.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [1]:
import pandas as pd
import numpy as np
import joblib
import gradio as gr
import os

# === Load assets ===
price_matrix = pd.read_csv("price_matrix.csv", index_col=0)
kmeans = joblib.load("KMeans_model.pkl")

# Transpose to match the ML model input: one row per coin
X = price_matrix.T  # shape: (n_coins, 30)
symbols = X.index.tolist()

# Predict cluster for each coin
clusters = kmeans.predict(X)

# Create DataFrame with symbol → cluster
cluster_df = pd.DataFrame({"symbol": symbols, "cluster": clusters})

# Compute daily returns and volatility
returns = price_matrix.pct_change().dropna()
avg_returns = returns.mean()
volatilities = returns.std()

# Store cluster statistics
cluster_stats = {}
for cluster_id in sorted(cluster_df["cluster"].unique()):
    coins_in_cluster = cluster_df[cluster_df["cluster"] == cluster_id]["symbol"]
    cluster_returns = avg_returns[coins_in_cluster]
    cluster_vols = volatilities[coins_in_cluster]
    
    cluster_stats[cluster_id] = {
        "mean_return": cluster_returns.mean(),
        "mean_volatility": cluster_vols.mean(),
        "coins": coins_in_cluster.tolist()
    }

def optimize_allocation(investment, objective):
    if investment <= 0:
        return "❌ Please enter a positive investment amount."

    if objective not in ["Maximum Profit", "Minimum Volatility"]:
        return "❌ Invalid objective. Choose one from the list."

    stats_df = pd.DataFrame(cluster_stats).T

    if objective == "Maximum Profit":
        scores = stats_df["mean_return"]
    else:
        scores = 1 / (stats_df["mean_volatility"] + 1e-8)

    weights = scores / scores.sum()
    allocations = weights * investment

    result_lines = [f"🎯 Optimization: {objective}", f"💰 Total Investment: ${investment:.2f}\n"]
    for cluster_id in stats_df.index:
        result_lines.append(f"🌀 Cluster {cluster_id}: ${allocations[cluster_id]:.2f}")

    return "\n".join(result_lines)

# === Gradio UI ===
demo = gr.Interface(
    fn=optimize_allocation,
    inputs=[
        gr.Number(label="💰 Investment Amount (USD)", precision=2),
        gr.Radio(label="🎯 Objective", choices=["Maximum Profit", "Minimum Volatility"])
    ],
    outputs=gr.Textbox(label="📊 Recommended Allocation per Cluster"),
    title="Crypto Investment Cluster Optimizer",
    description="Optimize your investment across crypto clusters for maximum return or minimum volatility.",
    allow_flagging="never"
)

demo.launch()


c:\Users\LENOVO\miniconda3\envs\dsi_participant\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\LENOVO\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
